# **Bank Customer Exit Predictor (CI PP-5)** 

# **Data Cleaning**

## Objectives

* Handle Missing Data
* Create Train and Test Datasets
* Clean data for analysis

## Inputs

* outputs/datasets/collection/BankCustomerData.csv

## Outputs

* Generate Train and Test datasets under : outputs/datasets/cleaned

## Conclusions

* Data Cleaning Pipeline
* No missing data
* Data is processed and ready for analysis

---

# Change working directory

* Notebooks are being stored in a subfolder, therefore when running the notebook in the editor, we need to change the working directory from its current folder to parent folder

1. We access the current directory with os.getcwd()

In [ ]:
import os
current_dir = os.getcwd()
current_dir

2. We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [ ]:
os.chdir(os.path.dirname(current_dir))
print("You have set a new current directory")

3. Confirm the new current directory

In [ ]:
current_dir = os.getcwd()
current_dir

---

# Load Data

* Loading dataset from outputs folder, however we are not including variables: CustomerID, Surname and RowNumber as they are just identifiers and dont impact the exit study.

In [ ]:
import pandas as pd
df = (pd.read_csv("outputs/datasets/collection/BankCustomerData.csv")
    .drop(['CustomerId','Surname','RowNumber'], axis=1)
    )
df.head(3)

---

# Data Exploration

* We check shape and distribution of a variables with missing data.

In [ ]:
missing_vars = df.columns[df.isna().sum() > 0].to_list() 
missing_vars

In [ ]:
from ydata_profiling import ProfileReport
if missing_vars:
    profile = ProfileReport(df=df[missing_vars], minimal=True)
    profile.to_notebook_iframe()
else:
    print("There are no variables with missing data")

---

# Correlation and PPS Analysis

* We do a correlation and PPS Analysis using a custom function which we studied in Code Intitute's course material. 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import ppscore as pps


def heatmap_corr(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=np.bool)
        mask[np.triu_indices_from(mask)] = True
        mask[abs(df) < threshold] = True

        fig, axes = plt.subplots(figsize=figsize)
        sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                    mask=mask, cmap='viridis', annot_kws={"size": font_annot}, ax=axes,
                    linewidth=0.5
                    )
        axes.set_yticklabels(df.columns, rotation=0)
        plt.ylim(len(df.columns), 0)
        plt.show()


def heatmap_pps(df, threshold, figsize=(20, 12), font_annot=8):
    if len(df.columns) > 1:
        mask = np.zeros_like(df, dtype=np.bool)
        mask[abs(df) < threshold] = True
        fig, ax = plt.subplots(figsize=figsize)
        ax = sns.heatmap(df, annot=True, xticklabels=True, yticklabels=True,
                         mask=mask, cmap='rocket_r', annot_kws={"size": font_annot},
                         linewidth=0.05, linecolor='grey')
        plt.ylim(len(df.columns), 0)
        plt.show()


def CalculateCorrAndPPS(df):
    df_corr_spearman = df.corr(method="spearman")
    df_corr_pearson = df.corr(method="pearson")

    pps_matrix_raw = pps.matrix(df)
    pps_matrix = pps_matrix_raw.filter(['x', 'y', 'ppscore']).pivot(columns='x', index='y', values='ppscore')

    pps_score_stats = pps_matrix_raw.query("ppscore < 1").filter(['ppscore']).describe().T
    print("PPS threshold - check PPS score IQR to decide threshold for heatmap \n")
    print(pps_score_stats.round(3))

    return df_corr_pearson, df_corr_spearman, pps_matrix


def DisplayCorrAndPPS(df_corr_pearson, df_corr_spearman, pps_matrix, CorrThreshold, PPS_Threshold,
                      figsize=(20, 12), font_annot=8):

    print("\n")
    print("* Analyse how the target variable for your ML models are correlated with other variables (features and target)")
    print("* Analyse multi-colinearity, that is, how the features are correlated among themselves")

    print("\n")
    print("*** Heatmap: Spearman Correlation ***")
    print("It evaluates monotonic relationship \n")
    heatmap_corr(df=df_corr_spearman, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

    print("\n")
    print("*** Heatmap: Pearson Correlation ***")
    print("It evaluates the linear relationship between two continuous variables \n")
    heatmap_corr(df=df_corr_pearson, threshold=CorrThreshold, figsize=figsize, font_annot=font_annot)

    print("\n")
    print("*** Heatmap: Power Predictive Score (PPS) ***")
    print(f"PPS detects linear or non-linear relationships between two columns.\n"
          f"The score ranges from 0 (no predictive power) to 1 (perfect predictive power) \n")
    heatmap_pps(df=pps_matrix, threshold=PPS_Threshold, figsize=figsize, font_annot=font_annot)

* We calculate correlation and Predictive Power Score

In [ ]:
df_corr_pearson, df_corr_spearman, pps_matrix = CalculateCorrAndPPS(df)

* Display Heatmaps

In [ ]:
DisplayCorrAndPPS(df_corr_pearson = df_corr_pearson,
                  df_corr_spearman = df_corr_spearman, 
                  pps_matrix = pps_matrix,
                  CorrThreshold = 0.3, PPS_Threshold =0.01,
                  figsize=(12,10), font_annot=10)

---

# Data Cleaning

## Missing Data 

* We check missing data using a custom function 

In [ ]:
def EvaluateMissingData(df):
    missing_data_absolute = df.isnull().sum()
    missing_data_percentage = round(missing_data_absolute/len(df)*100, 2)
    df_missing_data = (pd.DataFrame(
                            data={"RowsWithMissingData": missing_data_absolute,
                                   "PercentageOfDataset": missing_data_percentage,
                                   "DataType": df.dtypes}
                                    )
                          .sort_values(by=['PercentageOfDataset'], ascending=False)
                          .query("PercentageOfDataset > 0")
                          )

    return df_missing_data

Checking missing data levels

In [ ]:
EvaluateMissingData(df)

## Data Cleaning Summary

### Drop Missing Data

* **Drop** - ['Age', 'Geography', 'HasCrCard', 'IsActiveMember'] (As the missing data level is not significant.)

In [ ]:
df.dropna(inplace=True)


Checking missing data level

In [ ]:
EvaluateMissingData(df)

## Split Train and Test Set 

In [ ]:
from sklearn.model_selection import train_test_split
TrainSet, TestSet, _, __ = train_test_split(
                                        df,
                                        df['Exited'],
                                        test_size=0.2,
                                        random_state=0)

print(f"TrainSet shape: {TrainSet.shape} \nTestSet shape: {TestSet.shape}")

Checking for missing data in Train and Test Set

In [ ]:
train_missing_data = EvaluateMissingData(TrainSet)
print(f"* There are {train_missing_data.shape[0]} variables with missing data \n")
train_missing_data


In [ ]:
test_missing_data = EvaluateMissingData(TestSet)
print(f"* There are {test_missing_data.shape[0]} variables with missing data \n")
test_missing_data

# Save Train and Test Datasets

In [ ]:
import os
try:
  os.makedirs(name='outputs/datasets/cleaned') # create outputs/datasets/collection folder
except Exception as e:
  print(e)

## Train Set

In [ ]:
TrainSet.to_csv("outputs/datasets/cleaned/TrainSet.csv", index=False)

## Test Set

In [ ]:
TestSet.to_csv("outputs/datasets/cleaned/TestSet.csv", index=False)

* Push the changes to GitHub Repo, using GitHub commands (git add, git commit, git push)